In [1]:
import numpy as np
import pandas as pd
import subprocess
import time

%matplotlib inline
import matplotlib.pyplot as plt 

In [2]:
import pyslha
import pyMDEO_LesHouches_generator

#Open xdict which is now a <PySLHA Doc: 10 blocks, 0 decays, 0 xsections>
xdict = pyMDEO_LesHouches_generator.buildSLHAinFile()

#Modified the xdict elements
#xdict.blocks['SPHENOINPUT'].entries[55]='%i           # Calculate loop corrected masses'%0

#Fermion matrix
Lam1 = 2.225000E-01 
Lam2 = 2.100000E-02
Lam3 = 3.000000E-02
Lam4 = 4.000000E-02
Lam7 = 3.000000E-02
Lam9 = 3.000000E-02
Lam10 = 1.000000E-02
vX = 5.2E3
VEV =246.220569
muC = 4.110000E+02 
MS2 = 4.020000E+08
Mn2 = 4.030000E+08

xdict.blocks['MINPAR'].entries[1]='%.6E    # lambda1Input'%Lam1
xdict.blocks['MINPAR'].entries[2]='%.6E    # lambda2Input'%Lam2
xdict.blocks['MINPAR'].entries[3]='%.6E    # lambda3Input'%Lam3
xdict.blocks['MINPAR'].entries[4]='%.6E    # lambda4Input'%Lam4
xdict.blocks['MINPAR'].entries[7]='%.6E    # lambda7Input'%Lam7
xdict.blocks['MINPAR'].entries[9]='%.6E    # lambda9Input'%Lam9
xdict.blocks['MINPAR'].entries[10]='%.6E    # lambda10Input'%Lam10
xdict.blocks['MINPAR'].entries[23]='%.6E    # mEt2Input'%Mn2
xdict.blocks['MINPAR'].entries[24]='%.6E    # MS2Input'%MS2
xdict.blocks['MINPAR'].entries[30]='%.6E    # vXinput'%vX
xdict.blocks['MINPAR'].entries[32]='%.6E    # muCinput'%muC

YX11 = 0.11
YX12 = 0.0011
YX21 = 0.02
YX22 = 0.32
xdict.blocks['YXIN'].entries[1,1]='%.6E    # YX(1,1)'%YX11
xdict.blocks['YXIN'].entries[1,2]='%.6E    # YX(1,2)'%YX12
xdict.blocks['YXIN'].entries[2,1]='%.6E    # YX(2,1)'%YX21
xdict.blocks['YXIN'].entries[2,2]='%.6E    # YX(2,2)'%YX22

#Write the Leshouches file
pyslha.writeSLHAFile('LesHouches.in.MDEO_low',xdict)

In [3]:
spheno = subprocess.getoutput('../.././SPheno-4.0.5/bin/SPhenoMDEO LesHouches.in.MDEO_low')
so = subprocess.getoutput('cat SPheno.spc.MDEO')

In [4]:
#ZL ad ZR matrices
ZL11 = eval(so.split('Block LEFTMIX')[1].split()[6])
ZL12 = eval(so.split('Block LEFTMIX')[1].split()[11])
ZL21 = eval(so.split('Block LEFTMIX')[1].split()[16])
ZL22 = eval(so.split('Block LEFTMIX')[1].split()[21])
print(ZL11,ZL12,ZL21,ZL22)

ZR11 = eval(so.split('Block RIGHTMIX')[1].split()[6])
ZR12 = eval(so.split('Block RIGHTMIX')[1].split()[11])
ZR21 = eval(so.split('Block RIGHTMIX')[1].split()[16])
ZR22 = eval(so.split('Block RIGHTMIX')[1].split()[21])
print(ZR11,ZR12,ZR21,ZR22)

#ZN etR,ss ->ns mixing
ZN11 = eval(so.split('Block NEUTRALMIX')[1].split()[6])
ZN12 = eval(so.split('Block NEUTRALMIX')[1].split()[11])
ZN21 = eval(so.split('Block NEUTRALMIX')[1].split()[16])
ZN22 = eval(so.split('Block NEUTRALMIX')[1].split()[21])
#print(ZN11,ZN12,ZN21,ZN22)

mXi_1 = eval(so.split('Fe_3')[1].split()[17])
mXi_2 = eval(so.split('Fe_3')[1].split()[21])
metI = eval(so.split('Fe_3')[1].split()[25])
metp = eval(so.split('Fe_3')[1].split()[29])
mns_1 = eval(so.split('Fe_3')[1].split()[33])
mns_2 = eval(so.split('Fe_3')[1].split()[37])
print(mXi_1,mXi_2,metI,metp,mns_1,mns_2)

-0.999605012 0.028103744 -0.028103744 -0.999605012
0.997406769 -0.0719703846 0.0719703846 0.997406769
403.284405 1179.33288 20085.0124 20084.9822 20053.2815 20085.0624


# Fermion diagonalization

In [5]:
def MATFermionXi(YX11,YX12,YX21,YX22,vX):
    
    dcOut={}
    
    MF = (-vX/np.sqrt(2.))*np.matrix([[YX11,YX12],[YX21,YX22]])

    (MVdiag2,V)=np.linalg.eig(MF*np.transpose(MF))

    (MUdiag2,U)=np.linalg.eig(np.transpose(MF)*MF)
    
    if np.sqrt(np.abs(MVdiag2[0])) < np.sqrt(np.abs(MVdiag2[1])):
        
        M1=np.sqrt(np.abs(MVdiag2[0]))
        M2=np.sqrt(np.abs(MVdiag2[1]))
        V11=V[0,0]
        V12=V[0,1]
        V21=V[1,0]
        V22=V[1,1]
        U11=U[0,0]
        U12=U[0,1]
        U21=U[1,0]
        U22=U[1,1]
        
    else:
        
        M1=np.sqrt(np.abs(MVdiag2[1]))
        M2=np.sqrt(np.abs(MVdiag2[0]))
        V11=V[1,0]
        V12=V[1,1]
        V21=V[0,0]
        V22=V[0,1]
        U11=U[1,0]
        U12=U[1,1]
        U21=U[0,0]
        U22=U[0,1]
    
    dcOut['M1']= M1
    dcOut['M2']= M2  
    dcOut['V11']= V11 
    dcOut['V12']= V12 
    dcOut['V21']= V21 
    dcOut['V22']= V22 
    dcOut['U11']= U11 
    dcOut['U12']= U12 
    dcOut['U21']= U21 
    dcOut['U22']= U22 
            
    return dcOut

In [6]:
MATFermionXi(YX11,YX12,YX21,YX22,vX)

{'M1': 403.2844051540688,
 'M2': 1179.3328825058381,
 'V11': -0.9996050117797957,
 'V12': -0.028103743962590196,
 'V21': 0.028103743962590196,
 'V22': -0.9996050117797957,
 'U11': -0.997406769444537,
 'U12': -0.07197038464682774,
 'U21': 0.07197038464682774,
 'U22': -0.997406769444537}

In [7]:
print(mXi_1,mXi_2)
print(ZL11,ZL12,ZL21,ZL22)
print(ZR11,ZR12,ZR21,ZR22)

403.284405 1179.33288
-0.999605012 0.028103744 -0.028103744 -0.999605012
0.997406769 -0.0719703846 0.0719703846 0.997406769


In [8]:
MXI = MATFermionXi(YX11,YX12,YX21,YX22,vX)

Check! Take care with the signs... working!!!

In [9]:
np.matrix([[MXI['V11']-ZL11, MXI['V12']+ZL12],[MXI['V21']+ZL21, MXI['V22']-ZL22]])

matrix([[ 2.20204299e-10,  3.74098044e-11],
        [-3.74098044e-11,  2.20204299e-10]])

# Scalar diagonalization

In [15]:
def MATSalarS0(l3,l4,l7,l9,l10,vX,VEV,muC,MS2,Mn2):
    
    dcOut={}
    
    MS = np.matrix([[0.5*((l3+l4)*VEV**2 + l7*vX**2) + Mn2, -0.5*VEV*muC],[-0.5*VEV*muC, 0.5*(l10*vX**2+l9*VEV**2) + MS2]])

    (MVdiag2,V)=np.linalg.eig(MS)
    
    if np.abs(MVdiag2[0]) < np.abs(MVdiag2[1]):
        
        M1=np.sqrt(np.abs(MVdiag2[0]))
        M2=np.sqrt(np.abs(MVdiag2[1]))
        V11=V[0,0]
        V12=V[0,1]
        V21=V[1,0]
        V22=V[1,1]
        
    else:
        
        M1=np.sqrt(np.abs(MVdiag2[1]))
        M2=np.sqrt(np.abs(MVdiag2[0]))
        V11=V[1,0]
        V12=V[1,1]
        V21=V[0,0]
        V22=V[0,1]
    
    dcOut['M1']= M1
    dcOut['M2']= M2  
    dcOut['V11']= V11 
    dcOut['V12']= V12 
    dcOut['V21']= V21 
    dcOut['V22']= V22 
            
    return dcOut

In [16]:
MATSalarS0(Lam3,Lam4,Lam7,Lam9,Lam10,vX,VEV,muC,MS2,Mn2)

{'M1': 20053.281507132742,
 'M2': 20085.062410261955,
 'V11': -0.03969656485854936,
 'V12': 0.9992117807244022,
 'V21': 0.9992117807244022,
 'V22': 0.03969656485854936}

In [12]:
print(mns_1,mns_2)
print(ZN11,ZN12,ZN21,ZN22)

20053.2815 20085.0624
-0.0396965649 -0.999211781 -0.999211781 0.0396965649


In [13]:
MSO = MATSalarS0(Lam3,Lam4,Lam7,Lam9,Lam10,vX,VEV,muC,MS2,Mn2)

In [14]:
np.matrix([[MSO['V11']-ZN11, MSO['V12']+ZN12],[MSO['V21']+ZN21, MSO['V22']-ZN22]])

matrix([[ 4.14506415e-11, -2.75597767e-10],
        [-2.75597767e-10, -4.14506415e-11]])

Check! Take care with the signs... working!!!